In [1]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import pickle

In [2]:
def get_infos(house_soup):
    house_info = dict()
    house_info['Location'] = house_soup.find(class_="line-clamp-2 underline lg:no-underline").get_text()
    house_info['Price'] = house_soup.find(class_="text-black-v8 text-xl font-semibold").get_text()
    house_info['Area'] = house_soup.find_all(class_="text-black-v8 text-xl font-semibold")[1].get_text()
    return house_info

In [3]:
page = 1
house_infos = list()
while True:
    url = requests.get("https://meeyland.com/cho-thue-nha-dat/phong-tro-da-nang?page=" + str(page))
    page_soup = bs(url.text, 'html.parser')
    try:
        page_soup.find(class_="text-black mt-5 text-sm").get_text() == "Không có tin đăng phù hợp" 
        break
    except: pass
    houses = page_soup.body.div.div.main.find_all(class_="h-12 mb-1 text-base font-medium line-clamp-2 break-word text-[#2b2b2b]")
    for i in range(len(houses)):
        houses[i] = "https://meeyland.com" + houses[i].find("a", href=True)['href']
    if (houses == []): break
    for house in houses:
        house_page = requests.get(house)
        house_soup = bs(house_page.text, 'html.parser')
        try: 
            house_infos.append(get_infos(house_soup))
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/meeyland.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/meeyland.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [3]:
with open("../Raw_Data/meeyland.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset

,Location,Price,Area
0,"Đ. Thanh Vinh 16, P. Hòa Khánh Bắc, Q. Liên Ch...","1,9 Triệu/Tháng",30 m²
1,"P. Hòa Minh, Q. Liên Chiểu, Tp. Đà Nẵng","1,8 Triệu/Tháng",29 m²
2,"P. Hòa Minh, Q. Liên Chiểu, Tp. Đà Nẵng",2 Triệu/Tháng,35 m²
3,"Da Nang Landlord, Q. Ngũ Hành Sơn, Tp. Đà Nẵng","5,5 Triệu/Tháng",45 m²
4,"Q. Sơn Trà, Tp. Đà Nẵng","1,8 Triệu/Tháng",18 m²
...,...,...,...
2217,"Q. Thanh Khê, Tp. Đà Nẵng","2,2 Triệu/Tháng",20 m²
2218,"Q. Cẩm Lệ, Tp. Đà Nẵng","3,5 Triệu/Tháng",25 m²
2219,"P. Hòa Xuân, Q. Cẩm Lệ, Tp. Đà Nẵng","2,16 Triệu/Tháng",24 m²
2220,"Vicoland, Đ. Tô Hiến Thành, Q. Sơn Trà, Tp. Đà...","5,04 Triệu/Tháng",28 m²


In [ ]:
dataset.to_csv('../Raw_Data/meeyland.csv')